## ......................................BASE DE DATOS DE CLIENTES..........................................................

## Este notebook genera la limpieza de las fuentes y genera un archivo consolidado para cada una de las fuentes que fue limpia.

###### Se importan librerias necesarias

In [1]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
matplotlib.style.use('ggplot')
import seaborn as sns
import os
from dateutil.parser import parse
pd.options.display.max_rows = 350
pd.options.display.max_columns = 350
from scipy import stats


##### Fuentes crm_clientes y eventos

In [2]:
#%cd /home/xubuntu/Documents/ProyectoFinalMINE/Datos/crm_eventos
%cd D:\MINE\ProyectoFinalMINE\Datos\crm_eventos
crm_eventos = pd.read_excel('eventos_del_crm.xlsx')

D:\MINE\ProyectoFinalMINE\Datos\crm_eventos


In [3]:
#%cd /home/xubuntu/Documents/ProyectoFinalMINE/Datos/crm_eventos
crm_clientes = pd.read_excel('crm_clientes.xlsx')

In [4]:
#%cd /home/xubuntu/Documents/ProyectoFinalMINE/Datos/Sector
%cd D:\MINE\ProyectoFinalMINE\Datos\Fuentes Usadas Tableau\entradas
sector_v1 = pd.read_excel('elsector.xlsx')

D:\MINE\ProyectoFinalMINE\Datos\Fuentes Usadas Tableau\entradas


## 1.Quitar columnas del crm_clientes

In [5]:
del crm_clientes["id"]
del crm_clientes["nombre"]
del crm_clientes["empresa"]
del crm_clientes["ip"]
del crm_clientes["navegador"]
del crm_clientes["Negociacion"]
del crm_clientes["Comentario inicial del cliente"]
del crm_clientes["Sector"]

In [6]:
crm_clientes.head(2)

,TipoUsuario,Prioridad,producto,ciudad,correo,correo.1,correo2,correoItcloud,corre,correo.2,Tipo Cotizacion a enviar,comercial,Fecha creacion,estado
0,Usuario_Final,4 Baja,Sms_Salida,Otra,0,carolina.arca88@gmail.com,0,0,NaN,NaN,SMS_Correo_CON_Precio,wrojas,2014-11-24 05:23:07,cerrado
1,Usuario_Final,3_Media,Sms_Salida,Armenia,0,gerencia@multitintas.com.co,0,0,NaN,NaN,Revendedor_SMS,wrojas,2014-11-24 05:28:08,cerrado


## 2. Traer del archivo “cliente_con_ventas_sector_eventos.xls” el sector del cliente. El cruce se hace por ID y de no encontrarse el ID se debe colocar “SinIdentificar”.

In [7]:
cliente_con_sector =  pd.merge(crm_clientes, sector_v1, how='left', left_on=['correoItcloud'],right_on=['correoItcloud'])

In [8]:
cliente_con_sector["Sector"]=cliente_con_sector["Sector"].fillna(0)

In [9]:
cliente_con_sector["Sector"]=cliente_con_sector["Sector"].apply(lambda x: "SinIdentificar" if x == 0 else x )

In [10]:
cliente_con_sector.tail(5)

,TipoUsuario,Prioridad,producto,ciudad,correo,correo.1,correo2,correoItcloud,corre,correo.2,Tipo Cotizacion a enviar,comercial,Fecha creacion,estado,Sector
2875,Usuario_Final,3_Media,Correo_Masivo,Bogota,1,info@gravitta.com,0,NaN,NaN,NaN,SMS_Correo_CON_Precio,ccastellanos,2018-02-08 13:33:37,En_Proceso,SinIdentificar
2876,Usuario_Final,1_SuperAlta,Sms_Salida,OTROS,1,ingenierotrejos@gmail.com,0,ingenierotrejos@gmail.com,0,0,SMS_Correo_CON_Precio,ccastellanos,2018-02-08 14:48:04,En_Proceso,SinIdentificar
2877,Usuario_Final,3_Media,Sms_Salida,Bogota,1,gerencia2@enlazar.net,0,gerencia2@enlazar.net,0,0,No_Enviar_Mail,ccastellanos,2018-02-08 15:59:30,cerrado,SinIdentificar
2878,Usuario_Final,1_SuperAlta,Sms_Salida,Bogota,1,juan@smartsi.info,0,NaN,NaN,NaN,SMS_Correo_CON_Precio,fpineda,2018-02-09 06:44:01,En_Proceso,SinIdentificar
2879,Usuario_Final,1_SuperAlta,Sms_Salida,Bogota,1,ssotec@hotmail.com,0,NaN,NaN,NaN,SMS_Correo_CON_Precio,fpineda,2018-02-09 07:06:29,En_Proceso,SinIdentificar


## 3. Quitar los espacios adelante y atrás de los campos CORREOITCLOUD, CORREO, CORREO2, CORRE y CORREO

In [11]:
cliente_con_sector['correo']=cliente_con_sector['correo'].apply(str)

In [12]:
cliente_con_sector['correo2']  =cliente_con_sector['correo2'].str.strip()
cliente_con_sector['corre']  =cliente_con_sector['corre'].str.strip()
cliente_con_sector['correoItcloud']  =cliente_con_sector['correoItcloud'].str.strip()
cliente_con_sector['correo']  =cliente_con_sector['correo'].str.strip()

In [13]:
cliente_con_sector.dtypes

TipoUsuario                         object
Prioridad                           object
producto                            object
ciudad                              object
correo                              object
correo.1                            object
correo2                             object
correoItcloud                       object
corre                               object
correo.2                            object
Tipo Cotizacion a enviar            object
comercial                           object
Fecha creacion              datetime64[ns]
estado                              object
Sector                              object
dtype: object

## 4. Crear el campo EMAIL y llenarlo con el resultado de la siguiente formula por cada registro

###### Si (correoItcloud<>"" y correoitcloud<>0 y correitcloud<>"." Y correoitcloud<>",") entonces correoitcloud

Sino Si (correo<>"" y correo<>0 y correo<>"." Y correo <>",") entonces correo

Sino Si (correo2<>"" y correo2<>0 y correo2<>"." Y correo2<>",") entonces correo2

Sino Si (corre<>"" y corre<>0 y corre<>"." Y corre<>",") entonces corre

Sino Si (correo<>"" y correo <>0 y correo<>"." Y correo<>",") entonces correo

Sino “TBD”

In [140]:
cliente_con_sector["correoItcloud"].isnull().sum()

0

In [141]:
cliente_con_sector["correoItcloud"]=cliente_con_sector["correoItcloud"].fillna(0)
cliente_con_sector['correo2']  =cliente_con_sector["correo2"].fillna(0)
cliente_con_sector['corre']  =cliente_con_sector["corre"].fillna(0)
cliente_con_sector['correo']  =cliente_con_sector["correo"].fillna(0)

In [142]:
#cliente_con_sector.loc[cliente_con_sector['correoItcloud'] == "nan"]
cliente_con_sector.head(2)

,TipoUsuario,Prioridad,producto,ciudad,correo,correo.1,correo2,correoItcloud,corre,correo.2,Tipo Cotizacion a enviar,comercial,Fecha creacion,estado,Sector,EMAIL,sinemail
0,Usuario_Final,4 Baja,Sms_Salida,Otra,0,carolina.arca88@gmail.com,0,0,0,NaN,SMS_Correo_CON_Precio,wrojas,2014-11-24 05:23:07,cerrado,SinIdentificar,0,TBN
1,Usuario_Final,3_Media,Sms_Salida,Armenia,0,gerencia@multitintas.com.co,0,0,0,NaN,Revendedor_SMS,wrojas,2014-11-24 05:28:08,cerrado,SinIdentificar,0,TBN


In [143]:
cliente_con_sector["correoItcloud"].head(10)

0                                      0
1                                      0
2    hozkarpatrick@funeraleslaaurora.com
3             ventas@itcloudcolombia.com
4                    gerencia@cvr.com.co
5                                      0
6               rita.arango@prays.com.co
7               sistemas@gruponavarro.co
8                                      0
9                                      0
Name: correoItcloud, dtype: object

In [144]:
def func_correo(x):
    if x == ",":
        return 0
    elif (x =="."):
        return 0
    elif x =="nan":
        return 0
    else:
        return x
    # Add elif statements....


In [182]:
def poner_EMAIL(x):
    if x["correoItcloud"] != 0 :
        return x["correoItcloud"]
    elif x["correo2"]  != 0:
        return x["correo2"]
    elif x["corre"]  != 0:
        return x["corre"]
    elif x["correo"]  != "0":
        return x["correo"]
    elif (x["correo"]  == "0") and (x["corre"]  == 0) and (x["correo2"]  == 0) and (x["correoItcloud"] == 0) :
        return x["sinemail"]
    else:
        return x["sinemail"]

In [184]:
cliente_con_sector["correoItcloud"]=cliente_con_sector["correoItcloud"].apply(func_correo)
cliente_con_sector['correo2']=cliente_con_sector["correo2"].apply(func_correo)
cliente_con_sector['corre']=cliente_con_sector["corre"].apply(func_correo)
cliente_con_sector["correo"]=cliente_con_sector["correo"].apply(func_correo)

In [185]:
cliente_con_sector["correo"]=cliente_con_sector["correo"].replace(['1', '2'], 0)

In [186]:
cliente_con_sector["EMAIL"]=None
cliente_con_sector["sinemail"]="TBN"

In [216]:
cliente_con_sector["EMAIL"]=cliente_con_sector.apply(poner_EMAIL, axis=1)

In [204]:
#solo se verifica la salida del filtro de los correos
salida=cliente_con_sector.loc[cliente_con_sector['correoItcloud'] == 0]
salida.loc[(salida['correo2'] != 0)]

,TipoUsuario,Prioridad,producto,ciudad,correo,correo.1,correo2,correoItcloud,corre,correo.2,Tipo Cotizacion a enviar,comercial,Fecha creacion,estado,Sector,EMAIL,sinemail,EMAIL_mayuscula
9,Usuario_Final,4 Baja,Correo_Masivo,Bogota,0,german.silva@clubecopetrol.com,yenny.ramirez@clubecopetrolbogota.com,0,0,NaN,No_Enviar_Mail,wrojas,2014-11-24 07:59:16,cerrado,SinIdentificar,yenny.ramirez@clubecopetrolbogota.com,TBN,<map object at 0x0000021E3F0D33C8>
18,Usuario_Final,4_Baja,Sms_Salida,Valledupar,0,jmvalera86@hotmail.com,ventas@merkagro.com.co,0,0,NaN,SMS_Correo_SIN_Precio,wrojas,2014-11-24 12:56:35,cerrado,SinIdentificar,ventas@merkagro.com.co,TBN,<map object at 0x0000021E3F0D33C8>
21,Usuario_Final,4_Baja,Sms_Salida,Bucaramanga,0,info.canaveral@cecam.edu.co,monica@cecam.edu.co,0,0,NaN,Revendedor_SMS,wrojas,2014-11-24 19:20:38,cerrado,SinIdentificar,monica@cecam.edu.co,TBN,<map object at 0x0000021E3F0D33C8>
44,Administrativo,4_Baja,W_Proveedor,Bogota,0,felipeenriquez@fymtech.com,leandroromero@fymtech.com,0,0,NaN,No_Enviar_Mail,wrojas,2014-12-06 14:16:18,En_Proceso,SinIdentificar,leandroromero@fymtech.com,TBN,<map object at 0x0000021E3F0D33C8>
45,Administrativo,4_Baja,W_Proveedor,Bogota,0,soporte@hostdime.com.co,client.ID.95820@hostdime.com,0,0,0,No_Enviar_Mail,wrojas,2014-12-06 14:26:14,En_Proceso,SinIdentificar,client.ID.95820@hostdime.com,TBN,<map object at 0x0000021E3F0D33C8>
67,Usuario_Final,2_Alta,Sms_Salida,Bogota,0,info@contactosolutions.com,ivan.bustamante@contactosolutions.com,0,0,NaN,Revendedor_SMS,wrojas,2014-12-17 13:46:28,cerrado,SinIdentificar,ivan.bustamante@contactosolutions.com,TBN,<map object at 0x0000021E3F0D33C8>
76,Usuario_Final,4_Baja,Sms_Salida,Bogota,0,ximena.cortes@grupoone.com,carlos.delvalle@grupoone.com,0,0,NaN,SMS_Correo_CON_Precio,wrojas,2014-12-18 17:30:57,cerrado,SinIdentificar,carlos.delvalle@grupoone.com,TBN,<map object at 0x0000021E3F0D33C8>
110,Usuario_Final,2_Alta,Sms_Salida,Bucaramanga,0,guillaume.morel@arvato.es,francisco.franco@arvato.es,0,0,NaN,SMS_Correo_CON_Precio,wrojas,2014-12-18 18:10:15,cerrado,SinIdentificar,francisco.franco@arvato.es,TBN,<map object at 0x0000021E3F0D33C8>
135,Usuario_Final,4 Baja,Marcacion_Automatica,Bogota,0,jorge.galat@ugc.edu.co,contador.kvd@gmail.com,0,0,NaN,Marcacion_Automatica_CON_Precio,wrojas,2014-12-22 15:36:23,cerrado,SinIdentificar,contador.kvd@gmail.com,TBN,<map object at 0x0000021E3F0D33C8>
138,Usuario_Final,3_Media,Marcacion_Automatica,Bogota,0,pablo922@hotmail.com,pc.combariza2656@uniandes.edu.co,0,0,0,SMS_Correo_CON_Precio,fpineda,2014-12-26 09:18:34,cerrado,SinIdentificar,pc.combariza2656@uniandes.edu.co,TBN,<map object at 0x0000021E3F0D33C8>


## 5. Pasar el campo EMAIL a mayúsculas

In [211]:
cliente_con_sector['EMAIL']=cliente_con_sector['EMAIL'].astype(str)

In [218]:
cliente_con_sector['EMAIL'] = cliente_con_sector['EMAIL'].str.upper()

In [220]:
del cliente_con_sector['EMAIL_mayuscula'] 
del cliente_con_sector['sinemail'] 

In [222]:
cliente_con_sector.head(3)

,TipoUsuario,Prioridad,producto,ciudad,correo,correo.1,correo2,correoItcloud,corre,correo.2,Tipo Cotizacion a enviar,comercial,Fecha creacion,estado,Sector,EMAIL
0,Usuario_Final,4 Baja,Sms_Salida,Otra,0,carolina.arca88@gmail.com,0,0,0,NaN,SMS_Correo_CON_Precio,wrojas,2014-11-24 05:23:07,cerrado,SinIdentificar,TBN
1,Usuario_Final,3_Media,Sms_Salida,Armenia,0,gerencia@multitintas.com.co,0,0,0,NaN,Revendedor_SMS,wrojas,2014-11-24 05:28:08,cerrado,SinIdentificar,TBN
2,Revendedor,3_Media,Sms_Salida,Manizales,0,sergio.arango@live.com,0,hozkarpatrick@funeraleslaaurora.com,0,NaN,Revendedor_SMS,wrojas,2014-11-24 06:30:10,cerrado,SinIdentificar,HOZKARPATRICK@FUNERALESLAAURORA.COM


## 6. Aplicar un formato sobre el campo PRIORIDAD para cambiar el valor “4 Baja” por el valor “4_Baja

In [223]:
cliente_con_sector.Prioridad.unique()

array(['4 Baja', '3_Media', '1_SuperAlta', '2_Alta', '4_Baja'], dtype=object)

In [224]:
cliente_con_sector["Prioridad"]=cliente_con_sector.Prioridad.replace(['4 Baja'],['4_Baja']) 

In [225]:
cliente_con_sector.Prioridad.unique()

array(['4_Baja', '3_Media', '1_SuperAlta', '2_Alta'], dtype=object)

## 7. Quitar los espacios adelante y atrás del campo CIUDAD

In [226]:
cliente_con_sector['solo_ciudad'] =cliente_con_sector['ciudad'].str.split(' - ').str.get(0)

In [227]:
del cliente_con_sector["ciudad"]

In [228]:
cliente_con_sector.rename(columns={'solo_ciudad': 'ciudad'}, inplace=True)

## 8. Aplicar un formato sobre el campo CIUDAD para cambiarlos a solo el nombre de la ciudad

In [230]:
cliente_con_sector["ciudad"]=cliente_con_sector["ciudad"].replace(['EspaÃ¯Â¿Â½a'], "España")

In [231]:
cliente_con_sector.ciudad.unique()

array(['Otra', 'Armenia', 'Manizales', 'Cali', 'Bogota', 'Pereira',
       'Neiva', 'Valledupar', 'Bucaramanga', 'Medellin', 'Barranquilla',
       'Cartagena', 'Ibague', 'Cucuta', 'barrancabermeja', 'OTROS',
       'Villavicencio', 'Tunja', 'Santa Marta', 'Venezuela', 'España'], dtype=object)

## 9. Crear el campo PERIODO con una combinación del año y el mes de la fecha de creación. Tener cuidado que el campo resultante debe ser de 6 dígitos

In [236]:
cliente_con_sector.head()

,TipoUsuario,Prioridad,producto,correo,correo.1,correo2,correoItcloud,corre,correo.2,Tipo Cotizacion a enviar,comercial,Fecha creacion,estado,Sector,EMAIL,ciudad
0,Usuario_Final,4_Baja,Sms_Salida,0,carolina.arca88@gmail.com,0,0,0,NaN,SMS_Correo_CON_Precio,wrojas,2014-11-24 05:23:07,cerrado,SinIdentificar,TBN,Otra
1,Usuario_Final,3_Media,Sms_Salida,0,gerencia@multitintas.com.co,0,0,0,NaN,Revendedor_SMS,wrojas,2014-11-24 05:28:08,cerrado,SinIdentificar,TBN,Armenia
2,Revendedor,3_Media,Sms_Salida,0,sergio.arango@live.com,0,hozkarpatrick@funeraleslaaurora.com,0,NaN,Revendedor_SMS,wrojas,2014-11-24 06:30:10,cerrado,SinIdentificar,HOZKARPATRICK@FUNERALESLAAURORA.COM,Manizales
3,Usuario_Final,1_SuperAlta,Sms_Salida,0,wrojasp@gmail.com,wrojasp@gmail.com,ventas@itcloudcolombia.com,prueba2@gmail.com,NaN,SMS_Correo_CON_Precio,wrojas,2014-11-24 07:26:29,cerrado,SinIdentificar,VENTAS@ITCLOUDCOLOMBIA.COM,Cali
4,Usuario_Final,2_Alta,Sms_Salida,0,comunicacionvisual@cvr.com.co,0,gerencia@cvr.com.co,0,0,SMS_Correo_CON_Precio,ccastellanos,2014-11-24 07:32:42,en_proceso_Con_Venta,Retail,GERENCIA@CVR.COM.CO,Bogota


In [242]:
cliente_con_sector['Fecha creacion ']=cliente_con_sector['Fecha creacion '].astype(str)

In [250]:
cliente_con_sector['Periodo'] =cliente_con_sector['Fecha creacion '].str.split('-').str.get(0)+cliente_con_sector['Fecha creacion '].str.split('-').str.get(1)

## 10.Calcular antiguedad

In [265]:
cliente_con_sector["Fecha creacion "] = pd.to_datetime(cliente_con_sector["Fecha creacion "])

In [285]:
cliente_con_sector["Antiguedad"]=cliente_con_sector["Fecha creacion "].apply(lambda x: datetime.now() - x)
cliente_con_sector['Antiguedad']=cliente_con_sector['Antiguedad'].astype(str)
cliente_con_sector['numerodias']=cliente_con_sector['Antiguedad'].str.split(' ').str.get(0)
cliente_con_sector['numerodias']=cliente_con_sector['numerodias'].astype(int)
cliente_con_sector['Antiguedad']=round(cliente_con_sector['numerodias']/30)

In [289]:
#del cliente_con_sector['numerodias']
del cliente_con_sector['antiguedad']

In [291]:
del cliente_con_sector['correo']
del cliente_con_sector['correo.1']
del cliente_con_sector['correo2']
del cliente_con_sector['correo.2']
del cliente_con_sector['correoItcloud']
del cliente_con_sector['corre']


In [293]:
del cliente_con_sector['año_periodo']

In [294]:
cliente_con_sector.head()

,TipoUsuario,Prioridad,producto,Tipo Cotizacion a enviar,comercial,Fecha creacion,estado,Sector,EMAIL,ciudad,Periodo,Antiguedad
0,Usuario_Final,4_Baja,Sms_Salida,SMS_Correo_CON_Precio,wrojas,2014-11-24 05:23:07,cerrado,SinIdentificar,TBN,Otra,201411,42.0
1,Usuario_Final,3_Media,Sms_Salida,Revendedor_SMS,wrojas,2014-11-24 05:28:08,cerrado,SinIdentificar,TBN,Armenia,201411,42.0
2,Revendedor,3_Media,Sms_Salida,Revendedor_SMS,wrojas,2014-11-24 06:30:10,cerrado,SinIdentificar,HOZKARPATRICK@FUNERALESLAAURORA.COM,Manizales,201411,42.0
3,Usuario_Final,1_SuperAlta,Sms_Salida,SMS_Correo_CON_Precio,wrojas,2014-11-24 07:26:29,cerrado,SinIdentificar,VENTAS@ITCLOUDCOLOMBIA.COM,Cali,201411,42.0
4,Usuario_Final,2_Alta,Sms_Salida,SMS_Correo_CON_Precio,ccastellanos,2014-11-24 07:32:42,en_proceso_Con_Venta,Retail,GERENCIA@CVR.COM.CO,Bogota,201411,42.0


## Se exporta el archivo final de los clientes del crm que incluyen el sector.

In [296]:
%cd D:\MINE\ProyectoFinalMINE\Datos\Fuentes Usadas Tableau\salidas
cliente_con_sector.to_excel("cliente_con_sector.xlsx")

D:\MINE\ProyectoFinalMINE\Datos\Fuentes Usadas Tableau\salidas
